In [2]:

import sys
sys.path.append('../../')
import pendulum
from config.HIVESERVER2_token import *
from pyutil.pysql.sql import SQLClient
import lightgbm as lgbm
import pandas as pd
import numpy as np
from icecream import ic

# 220627

In [2]:
query = '''
WITH label AS (
    SELECT 
        a.role_id , a.acc_id , a.label_dt
        , COALESCE(
            MAX(
                CASE
                WHEN DATE_DIFF('day',a.label_dt,b.purchase_dt) BETWEEN 0 AND 6 THEN b.future_7d_max_amount
                END
            ) 
            , 0
        ) AS max_price_next_week
    FROM (
        SELECT DISTINCT
            role_id
            , acc_id 
            , DATE(TRY_CAST(purchase_time AS TIMESTAMP)) AS label_dt
        FROM hive."10048_bi_dw".order_attr_all
        WHERE acc_id <> ''
            AND DATE(TRY_CAST(purchase_time AS TIMESTAMP)) BETWEEN DATE('{start_dt}') AND DATE('{end_dt}')
            AND recharge_type = 0
            AND COALESCE(refund_time,'') = ''
    ) AS a
    LEFT JOIN (
        SELECT 
            role_id
            , acc_id 
            , DATE(TRY_CAST(purchase_time AS TIMESTAMP)) AS purchase_dt
            , MAX(ROUND(t3.usd,2)) AS future_7d_max_amount
        FROM hive."10048_bi_dw".order_attr_all AS t2
        JOIN hive."10048_bi_dw".item_price AS t3
        ON t2.item = t3.item_id
        WHERE  acc_id <> ''
            AND DATE(TRY_CAST(purchase_time AS TIMESTAMP)) BETWEEN DATE('{start_dt}') AND DATE_ADD('day',7,'{end_dt}')
            AND recharge_type = 0
            AND COALESCE(refund_time,'') = ''
        GROUP BY role_id, acc_id , DATE(TRY_CAST(purchase_time AS TIMESTAMP)) 
    ) AS b
    ON a.role_id = b.role_id
        AND a.acc_id = b.acc_id
    GROUP BY a.role_id , a.acc_id , a.label_dt
)


SELECT 
    a.*, b.max_price_next_week 
FROM (
    SELECT *
    FROM hive."10048_ml_dw".bundle_price_role_common_feature_dd
    WHERE dt BETWEEN '{start_dt}' AND '{end_dt}'
        AND feature_group = 'attribute'
) AS a
JOIN label AS b
ON a.acc_id = b.acc_id
    AND a.role_id = b.role_id
    AND DATE(a.dt) = b.label_dt
'''

In [3]:
client = SQLClient('ethancao')
# today = pendulum.date(2023, 2, 12)

client.execute_trino_query('  select * from hive.ml_tmp_db.bundle_train_220627 ')
traindata = client.to_pandas()
print('Train Data Fetched')

Train Data Fetched


In [36]:
binarycol = []

for x in traindata:
    if traindata[x].value_counts().shape[0] <= 2:
        binarycol.append(x)



In [5]:

feature_name = 'acc_cost_usd,acc_first_purchase_price_usd,acc_role_cnt,acc_total_charge_cnt,acc_total_charge_sum_usd,acc_total_refund_cnt,acc_total_refund_sum_usd,bundle_pop_buy_count_last_day,bundle_pop_buy_count_last_day_campaign_stage_trigger,bundle_pop_buy_count_last_day_tech_unlock_trigger,bundle_pop_buy_count_last_week,bundle_pop_buy_count_last_week_adjust,bundle_pop_buy_max_last_day,bundle_pop_buy_max_last_week,bundle_pop_buy_max_last_week_adjust,bundle_pop_buy_max_price_campaign_stage_trigger,bundle_pop_buy_max_price_campaign_stage_trigger_commax,bundle_pop_buy_max_price_tech_unlock_trigger,bundle_pop_buy_max_price_tech_unlock_trigger_commax,bundle_pop_buy_sum_last_day,bundle_pop_buy_sum_last_week,bundle_pop_buy_sum_last_week_adjust,bundle_pop_count_campaign_stage_trigger_9999,bundle_pop_count_last_day,bundle_pop_count_last_day_campaign_stage_trigger,bundle_pop_count_last_day_tech_unlock_trigger,bundle_pop_count_last_week,bundle_pop_count_last_week_adjust,bundle_pop_count_tech_unlock_trigger_9999,buy_rate_last_day,buy_rate_last_day_campaign_stage_trigger,buy_rate_last_day_tech_unlock_trigger,buy_rate_last_week,if_accounts_first_role,if_pruchase_active_in_1day,if_pruchase_active_in_30day,if_pruchase_active_in_3day,if_pruchase_active_in_7day,is_acc_transfer_from_old,is_acc_transfer_to_new,is_role_transfer_from_old,is_role_transfer_to_new,is_valid,level,max_role_level,money_flow_in_cout_last_day,money_flow_in_cout_last_week,money_flow_in_cout_last_week_adjust,money_flow_out_cout_last_day,money_flow_out_cout_last_week,money_flow_out_cout_last_week_adjust,purchase_price_1499_last_week,purchase_price_1499_last_week_adjust,purchase_price_1999_last_week,purchase_price_1999_last_week_adjust,purchase_price_2999_last_week,purchase_price_2999_last_week_adjust,purchase_price_4999_last_week,purchase_price_4999_last_week_adjust,purchase_price_499_last_week,purchase_price_499_last_week_adjust,purchase_price_9999_last_week,purchase_price_9999_last_week_adjust,purchase_price_999_last_week,purchase_price_999_last_week_adjust,purchase_price_max,purchase_price_max_last_3days,purchase_price_max_last_day,purchase_price_max_last_month,purchase_price_max_last_week,purchase_price_max_last_week_adjust,purchase_price_sum_last_3days,purchase_price_sum_last_day,purchase_price_sum_last_month,purchase_price_sum_last_week,purchase_price_sum_last_week_adjust,purchase_price_sum_max_ratio,purchase_price_sum_max_ratio_com2,role_cost_usd,role_create_os_android,role_create_os_ios,role_first_purchase_price_usd,role_last_login_duration,role_last_purchase_price_usd,role_lifespan,role_login_last_now,role_login_secondlast_last,role_login_secondlast_now,role_logout_last_now,role_no,role_purchase_first_last,role_purchase_first_now,role_purchase_last_now,role_total_charge_cnt,role_total_charge_sum_usd,role_total_refund_cnt,role_total_refund_sum_usd,role_type_fraud,role_type_no_install_info,vip_level'
feature_name = feature_name.split(',')
label_dict = {0.99: 0, 1.99: 1, 2.99: 2, 4.99: 3, 9.99: 4, 14.99: 5, 19.99: 6, 29.99: 7, 49.99: 8, 99.99: 9}

ds = traindata.assign(
    max_price_next_week = lambda x: x.max_price_next_week.apply(lambda y: label_dict.get(round(y,2))) 
)

_ = ds.query(' dt <= "2023-01-20"  ')
trainset = lgbm.Dataset(_.loc[:,feature_name], label = _.max_price_next_week)


_ = ds.query(' dt > "2023-01-20"  ')
valset = lgbm.Dataset(_.loc[:,feature_name], label = _.max_price_next_week)


# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'metric': ['multi_logloss'],
#     'num_class': 10,
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': -100
# }

# gmb = lgbm.train(params, trainset, num_boost_round=150, feature_name = feature_name, valid_sets = [trainset,valset])



[1]	training's multi_logloss: 1.91725	valid_1's multi_logloss: 1.97359
[2]	training's multi_logloss: 1.86282	valid_1's multi_logloss: 1.92493
[3]	training's multi_logloss: 1.81729	valid_1's multi_logloss: 1.88413
[4]	training's multi_logloss: 1.77782	valid_1's multi_logloss: 1.84874
[5]	training's multi_logloss: 1.74323	valid_1's multi_logloss: 1.81833
[6]	training's multi_logloss: 1.71217	valid_1's multi_logloss: 1.79053
[7]	training's multi_logloss: 1.68455	valid_1's multi_logloss: 1.76601
[8]	training's multi_logloss: 1.65989	valid_1's multi_logloss: 1.74437
[9]	training's multi_logloss: 1.6377	valid_1's multi_logloss: 1.72481
[10]	training's multi_logloss: 1.61777	valid_1's multi_logloss: 1.70732
[11]	training's multi_logloss: 1.59947	valid_1's multi_logloss: 1.69177
[12]	training's multi_logloss: 1.58278	valid_1's multi_logloss: 1.67747
[13]	training's multi_logloss: 1.56761	valid_1's multi_logloss: 1.66456
[14]	training's multi_logloss: 1.55371	valid_1's multi_logloss: 1.65271
[1

In [6]:
# val
query = '''
WITH bundle_pop_data AS (
    SELECT *
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY user_id, open_id, bundle_id, trace_id ORDER BY event_time DESC) AS rnk
        FROM hive."10048_oss_bi_dw".wgame_trigger_bundle_v2
        WHERE dt BETWEEN '{start_dt}' AND '{end_dt}'
            AND action = 'campaign_stage_trigger'
            AND trigger_type LIKE '%advancecard%'
    ) AS tmp
    WHERE rnk = 1
)

, bundle_buy_data AS (
    SELECT *
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY user_id, open_id, bundle_id, trace_id ORDER BY event_time DESC) AS rnk
        FROM hive."10048_oss_bi_dw".wgame_trigger_bundle_v2
        WHERE dt BETWEEN '{start_dt}' AND '{end_dt}'
            AND action = 'spc_bundle_buy'
            AND trigger_type LIKE '%advancecard%'
    ) AS tmp
    WHERE rnk = 1
)

, bundle_label AS (
    SELECT 
        a.*
        , IF(b.bundle_id IS NOT NULL,1,0) AS if_buy_pop_gift 
    FROM bundle_pop_data AS a
    LEFT JOIN bundle_buy_data AS b
    ON a.open_id = b.open_id
        AND a.user_id = b.user_id
        AND a.trace_id = b.trace_id
        AND a.bundle_id = b.bundle_id
)


, b AS (
    SELECT 
        CRC32(TO_UTF8(CONCAT_WS('@', a.open_id, a.user_id, a.bundle_id, a.trace_id, a.dt))) AS sampleid
        , b.feature AS feature
        , a.if_buy_pop_gift
        , a.bundle_price
        , ROW_NUMBER() OVER (PARTITION BY a.open_id, a.user_id, a.bundle_id, a.trace_id, a.dt ORDER BY a.event_time) As rnk
    FROM bundle_label AS a
    JOIN (
        SELECT *
        FROM hive."10048_ml_dw".bundle_price_role_common_feature_dd
        WHERE dt BETWEEN '2023-02-03' AND '2023-02-11'
            AND feature_group = 'attribute'
    ) AS b
    ON a.open_id = b.acc_id
        AND a.user_id = b.role_id
        AND a.dt = b.dt
)

SELECT *
FROM b
WHERE rnk = 1

'''
today = pendulum.date(2023, 2, 12)
valdata = []

for day in range(1,10):
    params = dict(
        start_dt=today.add(days=-day).to_date_string(),
        end_dt=today.add(days=-day).to_date_string(),
    )
    client.execute_trino_query(query.format(**params))
    valdata.append(client.to_pandas())


In [15]:

v = pd.concat(valdata, ignore_index=True).reset_index(drop = True)
inverse_label_dict = {v:k for k,v in label_dict.items()}
_ = v.assign(
    bundle_price = lambda x: x.bundle_price.apply(lambda y: int(y) /100 ),
    feature = lambda x: x.feature.apply(lambda y: [[_.split('\x03')[0], float(_.split('\x03')[1])] for _ in y.split('\x02') if _.split('\x03')[0] in feature_name])
).explode('feature').assign(
    col = lambda x: x.feature.apply(lambda y:y[0]),
    value = lambda x: x.feature.apply(lambda y:y[1]),
).pivot(
    index = ['sampleid','if_buy_pop_gift','bundle_price'], columns=['col'], values=['value'],
).value.reset_index()

ypred = gmb.predict(_.loc[:,feature_name])
ypred = np.argmax(ypred, axis = 1)

def tmpfunc(x):
    if x<=19.99:return 19.99
    elif x <= 49.99: return 49.99
    else: return 99.99

ypred = [inverse_label_dict.get(p) for p in ypred]
ypred = [tmpfunc(p) for p in ypred]
v['pred_bundle_price'] = ypred

In [17]:
v.pred_bundle_price.value_counts()

19.99    13723
99.99     1059
49.99      793
Name: pred_bundle_price, dtype: int64

In [19]:
badcase = v.assign(
    bundle_price = lambda x: x.bundle_price.apply(lambda y: int(y) /100 ),
).query(' bundle_price > pred_bundle_price ').query(' if_buy_pop_gift > 0 ')
badcase_rate = '{:.2%}'.format(badcase.shape[0] / v.shape[0])
badcase_loss = badcase.bundle_price.sum() - badcase.pred_bundle_price.sum()

greycase = v.assign(
    bundle_price = lambda x: x.bundle_price.apply(lambda y: int(y) /100 ),
).query(' bundle_price < pred_bundle_price ').query(' if_buy_pop_gift == 0 ')
greycase_rate = '{:.2%}'.format(greycase.shape[0] / v.shape[0])

ic(badcase_rate, badcase_loss, greycase_rate)

ic| badcase_rate: '3.67%'
    badcase_loss: 40770.0
    greycase_rate: '9.96%'


('3.67%', 40770.0, '9.96%')

# 220823p2

In [12]:
from sklearn.preprocessing import OneHotEncoder
from scipy.special import softmax


def get_true_price(v):
    inverse_label_dict = {0: 499, 1: 999, 2: 1499, 3: 1999, 4: 2999, 5: 4999, 6: 9999}
    return inverse_label_dict[v]

def custom_f_test(y_pred, train_data):
    get_true_price_func = lambda x: get_true_price(x)
    vf = np.vectorize(get_true_price_func)
    labels = train_data.get_label()
    preds = np.reshape(y_pred, (len(labels), 7))
    preds_softmax = softmax(preds, axis=1)
    preds_softmax_arg_max = np.argmax(preds_softmax, axis=1)
    pred_price = vf(preds_softmax_arg_max)
    true_price = vf(labels)
    penalty = (np.round(np.where(true_price >= pred_price, true_price - pred_price, true_price)))
    labels_expand = OneHotEncoder(sparse=False).fit_transform(labels.reshape(-1, 1))
    penalty_reshape = penalty.reshape((len(labels), 1))
    grad = (preds_softmax - labels_expand)  # turn around
    grad_delta = penalty_reshape * grad
    hess = 2.0 * preds_softmax * (1.0 - preds_softmax)
    return grad_delta.flatten("F"), hess.flatten("F")

In [22]:
client = SQLClient('ethancao')
today = pendulum.date(2023, 2, 12)

# 训练当天这个表
client.execute_trino_query('  select * from hive.ml_tmp_db.bundle_train220823 where if_buy_pop_gift >0')
traindata = client.to_pandas()
print('Train Data Fetched')

Train Data Fetched


In [31]:

feature_name = 'role_cost_usd,role_lifespan,role_no,is_role_transfer_to_new,is_role_transfer_from_old,role_purchase_first_now,role_first_purchase_price_usd,role_login_last_now,role_login_secondlast_now,role_login_secondlast_last,role_total_charge_sum_usd,role_total_charge_cnt,role_total_refund_sum_usd,role_total_refund_cnt,level,vip_level,mock_type,role_purchase_last_now,role_purchase_first_last,role_last_purchase_price_usd,is_valid,role_logout_last_now,role_last_login_duration,bundle_pop_count_last_day,bundle_pop_count_last_week,bundle_pop_count_last_day_campaign_stage_trigger,bundle_pop_count_last_day_officer_lv_trigger,bundle_pop_count_last_day_tech_unlock_trigger,bundle_pop_count_officer_lv_trigger_9999,bundle_pop_count_tech_unlock_trigger_9999,bundle_pop_count_campaign_stage_trigger_9999,bundle_pop_buy_count_last_day,bundle_pop_buy_count_last_week,bundle_pop_buy_sum_last_day,bundle_pop_buy_sum_last_week,bundle_pop_buy_max_last_day,bundle_pop_buy_max_last_week,bundle_pop_buy_count_last_day_campaign_stage_trigger,bundle_pop_buy_count_last_day_tech_unlock_trigger,bundle_pop_buy_count_last_day_officer_lv_trigger,bundle_pop_buy_max_price_campaign_stage_trigger,bundle_pop_buy_max_price_officer_lv_trigger,bundle_pop_buy_max_price_tech_unlock_trigger,purchase_price_max,purchase_price_max_last_day,purchase_price_max_last_3days,purchase_price_max_last_week,purchase_price_max_last_month,purchase_price_sum_last_day,purchase_price_sum_last_3days,purchase_price_sum_last_week,purchase_price_sum_last_month,purchase_price_499_last_week,purchase_price_999_last_week,purchase_price_1499_last_week,purchase_price_1999_last_week,purchase_price_2999_last_week,purchase_price_4999_last_week,purchase_price_9999_last_week,purchase_price_sum_max_ratio,purchase_price_sum_max_ratio_com2,if_pruchase_active_in_1day,if_pruchase_active_in_3day,if_pruchase_active_in_7day,if_pruchase_active_in_30day,bundle_pop_buy_max_price_campaign_stage_trigger_commax,bundle_pop_buy_max_price_officer_lv_trigger_commax,bundle_pop_buy_max_price_tech_unlock_trigger_commax,if_accounts_first_role,acc_cost_usd,acc_role_cnt,is_acc_transfer_to_new,is_acc_transfer_from_old,acc_first_purchase_price_usd,acc_total_charge_sum_usd,acc_total_charge_cnt,acc_total_refund_sum_usd,acc_total_refund_cnt,max_role_level,buy_rate_last_day_campaign_stage_trigger,buy_rate_last_day_tech_unlock_trigger,buy_rate_last_day,buy_rate_last_week,bundle_pop_count_last_week_adjust,bundle_pop_buy_count_last_week_adjust,bundle_pop_buy_sum_last_week_adjust,bundle_pop_buy_max_last_week_adjust,purchase_price_max_last_week_adjust,purchase_price_sum_last_week_adjust,purchase_price_499_last_week_adjust,purchase_price_999_last_week_adjust,purchase_price_1499_last_week_adjust,purchase_price_1999_last_week_adjust,purchase_price_2999_last_week_adjust,purchase_price_4999_last_week_adjust,purchase_price_9999_last_week_adjust,money_flow_out_cout_last_day,money_flow_out_cout_last_week,money_flow_in_cout_last_day,money_flow_in_cout_last_week,money_flow_out_cout_last_week_adjust,money_flow_in_cout_last_week_adjust'
feature_name = feature_name.split(',')
label_dict = {499: 0, 999: 1, 1499: 2, 1999: 3, 2999: 4, 4999: 5, 9999: 6}

ds = traindata.assign(
    bundle_price = lambda x:x.bundle_price.apply(lambda y : label_dict.get(int(y)))
)

_ = ds.query(' dt <= "2023-01-20"  ')
trainset = lgbm.Dataset(_.loc[:,feature_name], label = _.bundle_price)


_ = ds.query(' dt > "2023-01-20"  ')
valset = lgbm.Dataset(_.loc[:,feature_name], label = _.bundle_price)

params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': ['multi_logloss'],
        'num_class': 7,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }

gbm = lgbm.train(params,
                trainset,
                num_boost_round=150,
                feature_name=feature_name,
                fobj=custom_f_test,
                # categorical_feature=categorical_feature_name,
                valid_sets=[trainset,valset])


[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008739 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Using self-defined objective function
[1]	training's multi_logloss: 10.9302	valid_1's multi_logloss: 9.00173
[2]	training's multi_logloss: -2.51508	valid_1's multi_logloss: -2.17785
[3]	training's multi_logloss: -3.01209	valid_1's multi_logloss: -2.69997
[4]	training's multi_logloss: -3.30066	valid_1's multi_logloss: -2.97692
[5]	training's multi_logloss: -3.58695	valid_1's multi_logloss: -3.27645
[6]	training's multi_logloss: -4.08753	valid_1's multi_logloss: -3.66112
[7]	training's multi_logloss: -3.94839	valid_1's multi_logloss: -3.61838
[8]	training's multi_logloss: -3.99585	valid_1's multi_logloss: -3.65258
[9]	training's multi_logloss: -4.52089	valid_1's multi_logloss: -4.05499
[10]	training's multi_logloss: -4.61753	valid_1's multi_loglos

In [46]:

v = pd.concat(valdata, ignore_index=True).reset_index(drop = True)
inverse_label_dict = {v:k for k,v in label_dict.items()}
_ = v.assign(
    bundle_price = lambda x: x.bundle_price.apply(lambda y: int(y) /100 ),
    feature = lambda x: x.feature.apply(lambda y: [[_.split('\x03')[0], float(_.split('\x03')[1])] for _ in y.split('\x02') if _.split('\x03')[0] in feature_name])
).explode('feature').assign(
    col = lambda x: x.feature.apply(lambda y:y[0]),
    value = lambda x: x.feature.apply(lambda y:y[1]),
).pivot(
    index = ['sampleid','if_buy_pop_gift','bundle_price'], columns=['col'], values=['value'],
).value.reset_index()

ypred = gbm.predict(_.loc[:,feature_name])
ypred = np.argmax(ypred, axis = 1)

def tmpfunc(x):
    if x<=19.99:return 19.99
    elif x <= 49.99: return 49.99
    else: return 99.99

ypred = [inverse_label_dict.get(p) for p in ypred]
v['pred_bundle_price'] = [tmpfunc(p/ 100)  for p in ypred]

In [47]:
badcase = v.assign(
    bundle_price = lambda x: x.bundle_price.apply(lambda y: int(y) /100 ),
).query(' bundle_price > pred_bundle_price ').query(' if_buy_pop_gift > 0 ')
badcase_rate = '{:.2%}'.format(badcase.shape[0] / v.shape[0])
badcase_loss = badcase.bundle_price.sum() - badcase.pred_bundle_price.sum()

greycase = v.assign(
    bundle_price = lambda x: x.bundle_price.apply(lambda y: int(y) /100 ),
).query(' bundle_price < pred_bundle_price ').query(' if_buy_pop_gift == 0 ')
greycase_rate = '{:.2%}'.format(greycase.shape[0] / v.shape[0])

ic(badcase_rate, badcase_loss, greycase_rate)

ic| badcase_rate: '3.56%'
    badcase_loss: 39270.0
    greycase_rate: '13.71%'


('3.56%', 39270.0, '13.71%')

In [48]:
v.shape

(15575, 6)

In [44]:
set(ypred)

{499, 999, 1999, 4999, 9999}